In [1]:
# testing collab feature - marija
import ee
ee.Initialize(project='gist-itproject2')  

#NEW MESSAGE 
#CAN YOU SEE THIS
#omg yes this is so cool 
#better than word! also does running the code work 

In [2]:
# --- deps ---
import ee, json, os, zipfile, geopandas as gpd, pandas as pd
import geemap                     # for ee_to_geojson()
import geemap.foliumap as gf      # Folium-backed Map
import folium
import pandas as pd

ee.Initialize(project='gist-itproject2')

# -------------------------------
# 1) Indo-Pacific list (LSIB names)
# -------------------------------
ip_names = [
    "Afghanistan","Bangladesh","Bhutan","India","Maldives","Nepal","Pakistan","Sri Lanka",
    "Brunei","Burma","Cambodia","Indonesia","Laos","Malaysia","Philippines","Singapore",
    "Thailand","Timor-Leste","Vietnam",
    "China","Hong Kong","Macau","Japan","Korea, South","Korea, North","Mongolia","Taiwan",
    "Australia","Fiji","Kiribati","Marshall Islands","Micronesia, Federated States of","Nauru",
    "New Zealand","Palau","Papua New Guinea","Samoa","Solomon Islands","Tonga","Tuvalu","Vanuatu"
]

profiles = {
    "Australia":{"Capital":"Canberra","Ruling":"Australian Labor Party",
                 "Opposition":"Liberal-Nationals Coalition",
                 "Religion1":"No Religion (38.7%)","Religion2":"Catholic (19.6%)"},
    "China":{"Capital":"Beijing","Ruling":"Chinese Communist Party (CCP)",
             "Opposition":"None (one-party state)",
             "Religion1":"Unaffiliated (52.1%)","Religion2":"Chinese Folk Religion (21.9%)"},
    "India":{"Capital":"New Delhi","Ruling":"BJP-led coalition","Opposition":"INDIA bloc",
             "Religion1":"Hinduism (79.8%)","Religion2":"Islam (14.2%)"},
    "Indonesia":{"Capital":"Nusantara (transitional)","Ruling":"Gerindra (President)",
                 "Opposition":"PDI-P (role in flux)",
                 "Religion1":"Muslim (87.2%)","Religion2":"Protestant (7%)"},
    "Japan":{"Capital":"Tokyo","Ruling":"LDP–Komeito coalition","Opposition":"CDP",
             "Religion1":"Shinto (48.6%)","Religion2":"Buddhism (46.4%)"},
    "Malaysia":{"Capital":"Kuala Lumpur","Ruling":"Pakatan Harapan (PH)",
                "Opposition":"Perikatan Nasional (PN)",
                "Religion1":"Islam (63.5%)","Religion2":"Buddhism (18.7%)"},
    "New Zealand":{"Capital":"Wellington","Ruling":"National-led coalition","Opposition":"Labour",
                   "Religion1":"No Religion (51.6%)","Religion2":"Christianity (32.3%)"},
    "Philippines":{"Capital":"Manila","Ruling":"Alyansa para sa Bagong Pilipinas",
                   "Opposition":"DuterTen coalition",
                   "Religion1":"Roman Catholic (78.8%)","Religion2":"Islam (6.4%)"},
    "Korea, South":{"Capital":"Seoul","Ruling":"People Power Party (PPP)",
                    "Opposition":"Democratic Party (DP)",
                    "Religion1":"Unaffiliated (46.5%)","Religion2":"Christian (29%)"},
    "Thailand":{"Capital":"Bangkok","Ruling":"Pheu Thai coalition",
                "Opposition":"People's Party (formerly MFP)",
                "Religion1":"Theravada Buddhism (90–95%)","Religion2":"Muslim (~5%)"},
    "Vietnam":{"Capital":"Hanoi","Ruling":"Communist Party of Vietnam (CPV)",
               "Opposition":"None (one-party)",
               "Religion1":"Irreligion/Folk Religion (76.5%)","Religion2":"Buddhism (13.3%)"}
}

# -------------------------------
# 2) EE data & attributes
# -------------------------------
world = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
ip_fc = world.filter(ee.Filter.inList('country_na', ip_names))
non_ip_fc = world.filter(ee.Filter.Not(ee.Filter.inList('country_na', ip_names)))

def set_profile(f):
    name = ee.String(f.get('country_na'))
    prof = ee.Dictionary(profiles).get(name, ee.Dictionary({}))
    prof = ee.Dictionary(prof)
    return f.set({
        'Country': name,
        'Capital': prof.get('Capital', ''),
        'RulingParty': prof.get('Ruling', ''),
        'Opposition': prof.get('Opposition', ''),
        'Religion1': prof.get('Religion1', ''),
        'Religion2': prof.get('Religion2', '')
    })

ip_fc = ip_fc.map(set_profile)

# diamond centroids (one per country)
centroids = (
    ip_fc
    .map(lambda f: ee.Feature(f.geometry().centroid(1), f.toDictionary()))
    .distinct('Country')
)

# -------------------------------
# 3) Folium map
# -------------------------------
m = gf.Map(center=[-10, 140], zoom=3)
m.add_basemap("CartoDB.Positron")

# EE styled overlays
non_ip_style = non_ip_fc.style(color="d0d0d0", fillColor="5ca2ff", width=0.2)
ip_fill      = ip_fc.style(color="ffffff", fillColor="ffcc00", width=1.1)
ip_outline   = ip_fc.style(color="ffffff", fillColor="00000000", width=2.0)

m.addLayer(non_ip_style, {}, "Outside Indo-Pacific", True, 0.18)
m.addLayer(ip_fill, {}, "Indo-Pacific (fill)", True, 0.95)
m.addLayer(ip_outline, {}, "Indo-Pacific (outline)", True, 1.0)

# -------------------------------
# 4) Country diamond markers
# -------------------------------
gj = geemap.ee_to_geojson(centroids)

diamond_icon_html = """
<div style="width:14px;height:14px;background:#ffcc00;border:2px solid white;
            transform: rotate(45deg);box-shadow:0 0 3px rgba(0,0,0,.35);
            border-radius:1px;"></div>"""

marker_group = folium.FeatureGroup(name="Country markers", show=True)

for feat in gj["features"]:
    lon, lat = feat["geometry"]["coordinates"]
    p = feat["properties"]

    popup_html = f"""
    <div style="font-family:system-ui;min-width:240px;line-height:1.35">
      <div style="font-weight:700;margin-bottom:4px">{p.get('Country','')}</div>
      <div><b>Capital:</b> {p.get('Capital','') or '—'}</div>
      <div><b>Ruling:</b> {p.get('RulingParty','') or '—'}</div>
      <div><b>Opposition:</b> {p.get('Opposition','') or '—'}</div>
      <div><b>Religions:</b> {p.get('Religion1','') or '—'}{', ' if p.get('Religion2') else ''}{p.get('Religion2','')}</div>
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=280),
        icon=folium.DivIcon(html=diamond_icon_html)
    ).add_to(marker_group)

marker_group.add_to(m)

# -------------------------------
# 5) Volcanoes (Red Triangles) from CSV
# -------------------------------

volcano_df = pd.read_csv("GVP_Volcano_List_Holocene_202504090341_edit.csv")

# Filter only rows with valid coordinates
volcano_df = volcano_df[volcano_df['Latitude'].notnull() & volcano_df['Longitude'].notnull()]


volcano_layer = folium.FeatureGroup(name="Volcanoes", show=False)

# Volcano type to color mapping
type_colors = {
    "Stratovolcano": "red",
    "Shield volcano": "orange",
    "Caldera": "purple",
    "Lava dome": "blue",
    "Complex volcano": "green",
    "Pyroclastic cone": "pink",
    "Submarine volcano": "cyan",
    "Maar": "brown"
}
# Add volcano markers
for _, row in volcano_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    vtype = row.get('Primary Volcano Type', '')
    color = type_colors.get(vtype, 'gray')  # fallback if type missing

    triangle_icon_html = f"""
    <div style="
      width: 0;
      height: 0;
      border-left: 7px solid transparent;
      border-right: 7px solid transparent;
      border-bottom: 14px solid {color};
      transform: rotate(0deg);
    "></div>
    """

    popup_html = f"""
    <div style="font-family:system-ui;min-width:240px;line-height:1.35">
      <div style="font-weight:700;margin-bottom:4px">{row.get('Volcano Name','')}</div>
      <div><b>Country:</b> {row.get('Country','—')}</div>
      <div><b>Last Eruption:</b> {row.get('Last Known Eruption','—')}</div>
      <div><b>Elevation:</b> {row.get('Elevation (m)','—')} m</div>
      <div><b>Type:</b> {vtype or '—'}</div>
      <div><b>Rock Type:</b> {row.get('Dominant Rock Type','—')}</div>
      <div><b>Tectonic Setting:</b> {row.get('Tectonic Setting','—')}</div>
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.DivIcon(html=triangle_icon_html)
    ).add_to(volcano_layer)
# Add volcano layer to map
volcano_layer.add_to(m)

# -------------------------------
# 6) Final controls and display
# -------------------------------
folium.LayerControl(collapsed=False).add_to(m)

# Show in notebook
m

# Optional: Save as standalone HTML
# m.save("indo_pacific_map_with_volcanoes.html")
# print("Saved: indo_pacific_map_with_volcanoes.html")

In [3]:
#m.to_html("MainMap.html")